In [1]:
import json
import jsonlines
import sys
import prompt_rag

search_engine = "brave"

with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search.json", "r") as f:
    data_search = json.load(f)

try:
    with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search_llm.json", "r") as f:
        data_search_llm = json.load(f)
except:
    data_search_llm = [{
        "claim": i["claim"],
        "claimant": i["claimant"],
        "label": i["label"],
        "date": i["date"],
    } for i in data_search]
    with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search_llm.json", "w") as f:
        json.dump(data_search_llm, f, indent=4)


In [2]:
# prompt_rag.update_train_search_llm( 
#     "SOLAR-10.7B-Instruct-v1.0", "solar", port=8001, 
#     data_search=data_search, data_search_llm=data_search_llm) 



合并多个文件的先验知识

In [3]:
# # 对prior_knowledge做修改
# sort = False
# prior_knowledge_list = []

# K = 5
# prior_knowledge_version = "1"
# prior_knowledge_version_adjust = prior_knowledge_version + ".2"
# model_name = "mixtral"


# for i, item in enumerate(data_search_llm):
#     prior_knowledge = item[f"prior_knowledge_{model_name}_v{prior_knowledge_version}_K={K}"]
    
#     find_list = [
#         "Information Expansion and Summary:",
#         # "Information Summary:",
#         # "Expanded Information Summary:",
#         "Claim Analysis:",
#         # "Claim Verification",
#         # "Claim Analysis and Judgement:"
#     ]


#     for s in find_list:
#         pos = prior_knowledge.find(s)
#         if pos == -1:
#             continue
        
#         num_extra = 2
#         if pos!=0 and prior_knowledge[pos-1] == '\n':
#             pos -= 1
#             num_extra += 1
#         prior_knowledge = prior_knowledge[:pos] + prior_knowledge[pos + len(s) + num_extra :]

  
#     item[f"prior_knowledge_{model_name}_v{prior_knowledge_version_adjust}_K={K}"] = prior_knowledge


In [4]:
# bing search llm
# v2: k = 5
# v3: 不对时间排序

# brave search llm

# Solar
# v1: k = 5
# v2: k = 5 随机选取

# Mixtral
# v1: k = 5
# v1.1: 删除结构化内容（Information Expansion and Summary、Claim Analysis）

sort = False
prior_knowledge_list = []

K = 5
prior_knowledge_version = "1"
model_name = "mixtral"


# with open(f"train_search_llm_tmp.json", "r") as f:
#     prior_knowledge_list = json.load(f)

# for i, item in enumerate(data_search_llm):
#     if item["claim"] != prior_knowledge_list[i]["claim"].strip():
#         print(i)
#         print(item["claim"])
#         print(prior_knowledge_list[i]["claim"])
#         raise Exception()
#     else:
#         # vv = ""
#         # if prior_knowledge_list[i].get(f"prior_knowledge_{model_name}3") is not None:
#         #     vv = "3"
#         # elif prior_knowledge_list[i].get(f"prior_knowledge_{model_name}2") is not None:
#         #     vv = "2"

#         item[f"prior_knowledge_{model_name}_v{prior_knowledge_version}_K={K}"] = prior_knowledge_list[i][f"prior_knowledge_{model_name}"]

# data_search_llm[0]


In [7]:
# prompt_rag.save_search_llm(data_search_llm, search_engine)


创建数据（带有先验知识的Prompt）以微调LLM

In [12]:
# dict_list = []

# data_version = "2"
# for i, item in enumerate(data_search_llm):
    
#     if int(item["label"]) != 1:
#         prompt = prompt_rag.get_prompt_with_prior_knowledge(
#             item["claim"], 
#             search_engine,
#             data_search[i][f"{search_engine}_search_results"], 
#             item[f"prior_knowledge_{model_name}_v{prior_knowledge_version}_K={K}"], 
#             K=K,
#             claim_date=item["date"],
#             sort=sort,
#             known_info=False, # 不含已知信息
#             ids=None 
#         )
#         label = "TRUE." if int(item["label"]) == 2 else "FALSE."
#         dict_list.append({"query": prompt, "response": label})
# print(dict_list[0]["query"])
# len(dict_list), dict_list[:2]


According to the CLAIM and the PRIOR KNOWLEDGE, please classify the CLAIM as TRUE or FALSE. If the content described by the CLAIM is correct, then classify it as TRUE; if the content described by the CLAIM is incorrect, then classify it as FALSE.

CLAIM:
Publication date: 2020-08-13
Content: Illinois Strengthens Face Mask Rules in Businesses

PRIOR KNOWLEDGE:
Information 1:
Publication date: 2013-08-06
Title: Illinois Strengthens Face Mask Rules in Businesses - WebMD
Content:
COVID-19 is a new type of coronavirus that causes mild to severe cases. Here’s a quick guide on how to spot symptoms, risk factors, prevent spread of the disease, and find out what to do if you think you have it.
Information 2:
Publication date: None
Title: press-release
Content:
Illinoisans can resume activities without wearing a mask indoors on February 28th except where required by federal, state, local, tribal, or territorial laws, rules, and regulations, including local business and workplace guidance. Federa

(12192,
 [{'query': 'According to the CLAIM and the PRIOR KNOWLEDGE, please classify the CLAIM as TRUE or FALSE. If the content described by the CLAIM is correct, then classify it as TRUE; if the content described by the CLAIM is incorrect, then classify it as FALSE.\n\nCLAIM:\nPublication date: 2020-08-13\nContent: Illinois Strengthens Face Mask Rules in Businesses\n\nPRIOR KNOWLEDGE:\nInformation 1:\nPublication date: 2013-08-06\nTitle: Illinois Strengthens Face Mask Rules in Businesses - WebMD\nContent:\nCOVID-19 is a new type of coronavirus that causes mild to severe cases. Here’s a quick guide on how to spot symptoms, risk factors, prevent spread of the disease, and find out what to do if you think you have it.\nInformation 2:\nPublication date: None\nTitle: press-release\nContent:\nIllinoisans can resume activities without wearing a mask indoors on February 28th except where required by federal, state, local, tribal, or territorial laws, rules, and regulations, including local bu

In [13]:
# LLM_dir = "/home/hanlv/workspace/code/research/infodemic/LLM/"
# with jsonlines.open(
#     LLM_dir + \
#     f"swift/examples/pytorch/llm/my_data/with_{model_name}_info/{search_engine}/data{data_version}.jsonl", mode="w") as file_jsonl:
#     for line in dict_list:
#         file_jsonl.write(line)


测试模型的先验知识生成效果：一次提问

In [8]:
import torch
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = '1,2'

dirs = ["../.."]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)

from swift.llm import (
    ModelType, get_vllm_engine, get_default_template_type,
    get_template, inference_vllm, VllmGenerationConfig
)
from custom import CustomModelType, CustomTemplateType

# model_type = ModelType.mixtral_moe_7b_instruct
model_type = CustomModelType.mixtral_moe_7b_instruct_gptq_int4
# model_type = CustomModelType.solar_instruct_10_7b

llm_engine = get_vllm_engine(
    model_type, 
    torch_dtype=torch.float16,  # 检查正确的数据类型！！！！
    tensor_parallel_size=2,
    model_id_or_path="/home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4",
    engine_kwargs = {
        # "enforce_eager": True,
        "max_num_seqs": 64,
        "seed": 42,
    }
)

template_type = get_default_template_type(model_type)
template = get_template(template_type, llm_engine.hf_tokenizer)

generation_config = VllmGenerationConfig(
    max_new_tokens=2048,
    temperature=0,
)

get_resp_list = lambda request_list : inference_vllm(
    llm_engine, template, request_list, 
    generation_config=generation_config, 
    use_tqdm=False,
)



2024-04-06 14:19:36,492 - modelscope - INFO - PyTorch version 2.1.2 Found.
2024-04-06 14:19:36,493 - modelscope - INFO - Loading ast index from /home/hanlv/.cache/modelscope/ast_indexer
2024-04-06 14:19:36,529 - modelscope - INFO - Loading done! Current index file version is 1.13.1, with md5 ad87f4a589251d86333765b92ab59e41 and a total number of 972 components indexed
[INFO:swift] Loading the model using model_dir: /home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4
[INFO:swift] Setting torch_dtype: torch.bfloat16
[INFO:swift] model_config: MixtralConfig {
  "_name_or_path": "/home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4",
  "architectures": [
    "MixtralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mixtral",
  "num_attention_heads": 32,
  "num_experts_per_tok": 2,
  "num_hidde

WARNING 04-06 14:19:37 config.py:618] Casting torch.bfloat16 to torch.float16.
WARNING 04-06 14:19:37 config.py:193] gptq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 04-06 14:19:37 config.py:433] Custom all-reduce kernels are temporarily disabled due to stability issues. We will re-enable them once the issues are resolved.


2024-04-06 14:19:40,467	INFO worker.py:1724 -- Started a local Ray instance.


INFO 04-06 14:19:41 llm_engine.py:87] Initializing an LLM engine with config: model='/home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4', tokenizer='/home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=2, disable_custom_all_reduce=True, quantization=gptq, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=42)
INFO 04-06 14:20:03 llm_engine.py:357] # GPU blocks: 3613, # CPU blocks: 4096
INFO 04-06 14:20:05 model_runner.py:684] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 04-06 14:20:05 model_runner.py:688] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_ut

In [9]:
K = 5
def get_id(claim):
    for i in range(len(data_search)):
        if claim.strip() in data_search[i]["claim"].strip():
            return i

i = 35
# i = get_id("An image shows a COVID-19 rail tanker.")

claim = data_search[i]["claim"]
search_results = data_search[i][f"{search_engine}_search_results"]

model_name = 'mixtral'
prompt_list = [
   prompt_rag.get_prompt_for_generating_prior_knowledge(
        claim, data_search[i]["date"], search_engine, search_results, model_name,
        K=K, sort=False, 
        # ids=data_search[i]["random_ids"],
        ids=None
    ),
]

request_list = [{'query': prompt[:-1]} for prompt in prompt_list]

print(prompt_list[0])

resp_list = get_resp_list(request_list)
print(resp_list[0]["response"].strip())
print()
resp_list[0]["response"].strip()


Below is a CLAIM and some INFORMATION searched online. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first provide a detailed summary of the given INFORMATION. Then reason, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge. In reasoning, it is necessary to consider the sequential relationship between the date of publication of the CLAIM and the date of publication of the INFORMATION.

CLAIM:
Publication date: 2020-07-20
Content: Multiple Facebook posts shared tens of thousands of times claim the novel coronavirus, which causes the disease COVID-19, has infected far fewer people than the H1N1 virus, or swine fl

Information Summary:

Information 1, published on 2020-08-03, mentions that multiple Facebook posts claim COVID-19 has infected fewer people than H1N1 and that media hysteria has led to an overblown response. This information states that this claim is misleading and that COVID-19 is 10 times more deadly than swine flu.

Information 2, no publication date, describes a viral image comparing the number of infections of COVID-19 and H1N1, suggesting that the reaction to COVID-19 is overblown.

Information 3, no publication date, discusses the concept of "hysteria" and mentions COVID-19 as one of the hysterias experienced currently.

Information 4, published on 2020-03-20, clarifies that the comparison of infection and death numbers for H1N1 and COVID-19 is misleading, as H1N1 data was collected over a year, while COVID-19 data is from a shorter timeframe. It also mentions that the panic level during H1N1 was not "totally chill" and that media attention was immense.

Information 5, publishe

'Information Summary:\n\nInformation 1, published on 2020-08-03, mentions that multiple Facebook posts claim COVID-19 has infected fewer people than H1N1 and that media hysteria has led to an overblown response. This information states that this claim is misleading and that COVID-19 is 10 times more deadly than swine flu.\n\nInformation 2, no publication date, describes a viral image comparing the number of infections of COVID-19 and H1N1, suggesting that the reaction to COVID-19 is overblown.\n\nInformation 3, no publication date, discusses the concept of "hysteria" and mentions COVID-19 as one of the hysterias experienced currently.\n\nInformation 4, published on 2020-03-20, clarifies that the comparison of infection and death numbers for H1N1 and COVID-19 is misleading, as H1N1 data was collected over a year, while COVID-19 data is from a shorter timeframe. It also mentions that the panic level during H1N1 was not "totally chill" and that media attention was immense.\n\nInformation 

In [9]:
model_type = CustomModelType.mixtral_moe_7b_instruct_gptq_int4
from swift.llm import get_model_tokenizer
model, tokenizer = get_model_tokenizer(
    model_type, 
    load_model=False
)


[INFO:swift] Loading the model using model_dir: /home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4
[INFO:swift] Setting torch_dtype: torch.bfloat16
[INFO:swift] model_config: MixtralConfig {
  "_name_or_path": "/home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4",
  "architectures": [
    "MixtralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mixtral",
  "num_attention_heads": 32,
  "num_experts_per_tok": 2,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "num_local_experts": 8,
  "output_router_logits": false,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "bits": 4,
    "damp_percent": 0.1,
    "desc_act": true,
    "group_size": 32,
    "model_file_base_name": "model",
    "model_name_or_path": null,
    "modules_in_block_to_quantize": [
      [

In [10]:
from tqdm.notebook import tqdm
data_search_llm = prompt_rag.load_search_llm_tmp()
K=5

for i in tqdm(range(len(data_search_llm))):
    item = data_search_llm[i]

    if item.get(f"prior_knowledge_mixtral3") is None:
        continue

    p1 = item["prior_knowledge_mixtral3"]
    if len(tokenizer(p1)["input_ids"]) >= 200:
        continue

    search_results = data_search[i][f"{search_engine}_search_results"]

    model_name = 'mixtral'
    claim = data_search[i]["claim"]
    prompt = prompt_rag.get_prompt_for_generating_prior_knowledge(
        claim, data_search[i]["date"], search_engine, search_results, model_name,
        K=K, sort=False, 
        ids=None
    )

    while len(tokenizer(p1)["input_ids"]) < 200:
        prompt = prompt[:-1]
        resp_list = get_resp_list([{'query': prompt}])
        p1 = resp_list[0]["response"].strip()
        
    item["prior_knowledge_mixtral3"] = p1
        
prompt_rag.save_search_llm_tmp(data_search_llm)


  0%|          | 0/14384 [00:00<?, ?it/s]

In [12]:
s = "The claim was published in 2020-06-26, stating that COVID-19 spread can be estimated by looking at past trends of influenza-like illnesses. To evaluate the claim, let's summarize the given information:\n\nInformation 1, published in 2024-01-05, discusses the weekly influenza surveillance report by CDC, including information about COVID-19, influenza, and RSV activity. The report highlights the importance of evaluating syndromic surveillance data in the context of other sources of surveillance data to obtain a more complete and accurate picture of respiratory virus activity.\n\nInformation 2, published in 2022-07-29, explains the differences between COVID-19 and the flu, including symptoms, complications, and the viruses that cause them.\n\nInformation 3, published in 2022-11-23, is the claim in question, which was originally published in 2020-06-26. It discusses estimating COVID-19 spread by looking at past trends of influenza-like illnesses.\n\nInformation 4, published in 2022-09-28, explains the similarities and differences between flu and COVID-19, including their causes, symptoms, and complications. It also mentions that COVID-19 spreads more easily than flu.\n\nInformation 5, without a publication date, discusses joint COVID-19 and influenza-like illness forecasts in the United States, including the use of ARGO-Nat and ARGOX-Joint-Ensemble models that combine Google search data, Influenza-like Illness activity data, and COVID-19 cases/deaths to produce estimates.\n\nBased on the available information, the claim (Information 3) was published in 2020-06-26, while more comprehensive information about COVID-19 and influenza-like illnesses was published later (Information 1, 2, 4, and 5). However, Information 5 does mention the use of Influenza-like Illness activity data in COVID-19 forecasts, which supports the idea of using past trends of influenza-like illnesses to estimate COVID-19 spread.\n\nConsidering the information available, the claim (Information 3) is partially supported, as it is plausible to use past trends of influenza-like illnesses to estimate COVID-19 spread. However, a more comprehensive understanding of both COVID-19 and influenza-like illnesses has been gained since the claim was published in 2020-06-26. Therefore, the claim should be interpreted with caution and in the context of more recent findings."

print(s)


The claim was published in 2020-06-26, stating that COVID-19 spread can be estimated by looking at past trends of influenza-like illnesses. To evaluate the claim, let's summarize the given information:

Information 1, published in 2024-01-05, discusses the weekly influenza surveillance report by CDC, including information about COVID-19, influenza, and RSV activity. The report highlights the importance of evaluating syndromic surveillance data in the context of other sources of surveillance data to obtain a more complete and accurate picture of respiratory virus activity.

Information 2, published in 2022-07-29, explains the differences between COVID-19 and the flu, including symptoms, complications, and the viruses that cause them.

Information 3, published in 2022-11-23, is the claim in question, which was originally published in 2020-06-26. It discusses estimating COVID-19 spread by looking at past trends of influenza-like illnesses.

Information 4, published in 2022-09-28, explains